# Atividade de Programação 02
## Percepção Computacional 2020.2e
### Prof. Herman

Utilizando a base de dados ["Free Spoken Digit Dataset (FSDD)"](https://github.com/Jakobovski/free-spoken-digit-dataset) e como inspiração o modelo de rede neural pré-existente para reconhecimento de dígitos falados disponível no  [github](https://github.com/adhishthite/sound-mnist), implemente um Notebook Python no Google Colab contemplando o seguinte:

1.  Reproduzir o experimento de treinamento/classificação de dígitos do código github mencionado acima. (2 pontos)
2.   Treinar e testar um classificador que, a partir de um arquivo .wav contendo o som de um dígito qualquer, identificar qual dos 6 voluntários da base de dados pronunciou aquele dígito. Garantir que os conjuntos de treinamento e de teste são disjuntos. Apresentar curvas de treinamento (validação e acurácia nos conjs. de treino  e de validação), o relatório de métricas no conj. de teste e a matriz de confusão. (3 pontos)
3.    Treinar e testar um classificador que, a partir de um arquivo .wav contendo o som de um dígito qualquer, identificar qual o sotaque (USA/neutral versus Outros) está presente na pronúncia do dígito. Apresentar os mesmos artefatos do item anterior. (3 pontos)
4.    Montar uma pequena amostra de dados com as vozes de pelo menos 2 voluntários da equipe pronunciando os mesmos dígitos da base de dados (3 gravações por dígito por voluntário) e avaliar os classificadores 1. e o 3 sobre esta amostra, focando apenas nos relatórios de classificação. (2 pontos)

Cada equipe tem total liberdade para:
* definir como irá tratar os dados (eventuais pré-processamentos) 
* definir quais características (se alguma) serão utilizadas para entrada dos modelos de classificação
* modificar o modelo existente ou escolher outro modelo de aprendizagem de máquina para as tarefas acima.

In [ ]:
#obtendo código-exemplo e base de dados num único comando 
!git clone https://github.com/adhishthite/sound-mnist.git

In [ ]:
#Base de dados completa
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git

## Imports necessários

In [ ]:
import keras
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

## Modelo da Rede Neural

Arquitetura utilizada no treinamento dos modelos das questões 1 e 2, apenas alterando a quantidade de classes na camada de saída.

A arquitetura consiste de 3 camadas convolucionais, 1 camada de max pooling, 3 camadas densas e a saída com ativação softmax. São feitas normalizações (batch normalization) após cada camada convolucional e densa, e também dropout pra cada camada densa.

In [ ]:
def get_cnn_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())

    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(BatchNormalization())

    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    return model

## Conversão de WAV para MFCC

Método para conversão de arquivos de áudio WAV para Mel-Frequency Cepstrum Coefficient (MFCC), que é uma representação do espectro de potência de um som.

O MFCC é utilizado para entrada das redes neurais do notebook.

In [ ]:
def wav2mfcc(file_path, max_pad_len=20):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=8000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

## Função de teste

In [ ]:
def check_preds(model, X, y):
    predictions = model.predict_classes(X)

    print(classification_report(y, to_categorical(predictions)))

## Função para visualizar gráficos do treinamento

In [ ]:
def plot_acc_val_graph(history):
  #  "Accuracy"
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()
  # "Loss"
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

## Função para plotar a matriz de confusão

In [ ]:
def plot_confusion_matrix(model, X_test, y_test, label):
  y_pred = model.predict_classes(X_test)

  cm = confusion_matrix(y_test.argmax(axis=1), y_pred)

  df_cm = pd.DataFrame(cm, index = label,
                    columns = label)
  plt.figure(figsize = (10,7))
  ax = sn.heatmap(df_cm, annot=True)
  ax.set_title('Confusion matrix')
  ax.set_ylabel("Label verdadeiro")
  ax.set_xlabel("Label predito")
  plt.show()

# Questão 1

Reproduzir o experimento de treinamento/classificação de dígitos do código github mencionado acima. 

## Carregando dados - Train/Test split

Os dados são carregados e convertidos para MFCC, além de armazenar o label do áudio através do nome do arquivo. (ex: **0**_nicolas_1.wav - nicolas falando o digito 0)

In [ ]:
def get_data_digit(recording_path):
    labels = []
    mfccs = []

    for f in os.listdir(recording_path):
        if f.endswith('.wav'):
            # MFCC
            mfccs.append(wav2mfcc(recording_path + f))

            # List of labels
            label = f.split('_')[0]
            labels.append(label)

    return np.asarray(mfccs), to_categorical(labels)

Adquire os dados, o modelo, e faz a separação em dados de treinamento e teste (90% treinamento e 10% teste).

In [ ]:
def get_all(recording_path):
    mfccs, labels = get_data_digit(recording_path)

    dim_1 = mfccs.shape[1]
    dim_2 = mfccs.shape[2]
    channels = 1
    classes = 10

    X = mfccs
    X = X.reshape((mfccs.shape[0], dim_1, dim_2, channels))
    y = labels

    input_shape = (dim_1, dim_2, channels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

    model = get_cnn_model(input_shape, classes)

    return X_train, X_test, y_train, y_test, model